In [28]:
import math
import os.path
import pickle
from os import path
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import scipy.stats
from scipy.stats import norm

In [144]:
file = 'test11.txt'
n_participants = 12
f = open(file, 'r')
all_values = []
for i, line in enumerate(f): 
    if i == 0:
        names = line.split('\t')
    elif i < n_participants+1:
        values = line.split('\t')
        all_values.append(values)

In [145]:
class object():
    def __init__(self, Name):
        self.Name = Name
        self.N = 0
        self.Mean = 0
        self.Sum = 0
        self.times_visited = 0
        
    def set_N(self, N):
        self.N += float(N)
        self.times_visited += 1
    
    def set_Mean(self, Mean):
        self.Mean += float(Mean)
        self.times_visited += 1
        
    def set_Sum(self, Sum):
        self.Sum += float(Sum)
        self.times_visited += 1
        
    def get_Name(self):
        return self.Name

In [146]:
def set_object(values, i, obj, object_info):
    if object_info == 'N':
        obj.set_N(values[i])
    elif object_info == 'Mean':
        obj.set_Mean(values[i])
    elif object_info == 'Sum':
        obj.set_Sum(values[i])
    return obj

def object_in_list(obj_list, object_name):
    for obj in obj_list:
        if obj.get_Name() == object_name:
            return True
    return False

all_info = []
for values in all_values:
    info_dict = {}
    for i, name in enumerate(names):
        name_list = name.split('_')
        if i > 0:
            page_list = name_list[1].split('/')

            # N/google/search/youtube/baldibale
            page_name = page_list[-1]
            # Rectangle 11
            object_name = name_list[-2]
            # N, Mean, Sum
            object_info = name_list[-1]
            # Time to first fixation
            info_type = name_list[0]

            # if page in dict
            if page_name in info_dict:
                # if type in dict[page]
                if info_type in info_dict[page_name]:
                    obj_list = info_dict[page_name][info_type]
                    # if there is a object with the same name
                    if object_in_list(obj_list, object_name):
                        for obj in obj_list:
                            if obj.get_Name() == object_name:
                                obj = set_object(values, i, obj, object_info)
                                break
                    # if there is no object with the same name
                    else:
                        obj = object(object_name)
                        obj = set_object(values, i, obj, object_info)
                        info_dict[page_name][info_type].append(obj)
                # if type not in dict[page]
                else:
                    obj = object(object_name)
                    obj = set_object(values, i, obj, object_info)
                    info_dict[page_name][info_type] = [obj]
            # if page not in dict
            else:
                obj = object(object_name)
                obj = set_object(values, i, obj, object_info)
                info_dict[page_name] = {info_type: [obj]}
    
    all_info.append(info_dict)

In [149]:
rect_info = {}
if path.exists('obj/' + "rectangle_info" + '.pkl'):
    print("LOADING PREVIOUS ANSWERS \n")
    with open('obj/' + "rectangle_info" + '.pkl', 'rb') as f:
        rect_info = pickle.load(f)
else:
    os.mkdir('obj/')

all_grouped_info = []
for info_dict in all_info:
    grouped_dict = {}
    for page in info_dict.keys():
        grouped_dict[page] = {}
        if page == 'B%20-%20patron%20saint%20of%20mental%20illness%20-%20Google%20Search.html?q=patron+saint+of+mental+illness&btnK=Google+Search (CRC)':
            titles, descriptions, verticals = 10, 20, 24
        elif page in rect_info:
            titles, descriptions, verticals = rect_info[page]
        else:
            titles = int(input("titles: "))
            descriptions = int(input("descriptions: "))
            verticals = int(input("verticals: "))
            rect_info[page] = [titles, descriptions, verticals]

        for info_type in info_dict[page].keys():
            grouped_dict[page][info_type] = {} 
            obj_list = info_dict[page][info_type] # list with objects
            title_list, descr_list, vert_list = [], [], []

            for obj in obj_list:
                name = obj.get_Name()
                num = int(name.split(' ')[1])
#                 if page == 'B%20-%20patron%20saint%20of%20mental%20illness%20-%20Google%20Search.html?q=patron+saint+of+mental+illness&btnK=Google+Search (CRC)':
#                     print(num)
                
                if titles >= num > 0:
                    title_list.append(obj)
                elif descriptions >= num > titles:
                    descr_list.append(obj)
                elif verticals >= num > descriptions:
#                     if page == 'B%20-%20patron%20saint%20of%20mental%20illness%20-%20Google%20Search.html?q=patron+saint+of+mental+illness&btnK=Google+Search (CRC)':
#                         print('dit is een vertical')
                    vert_list.append(obj)
                elif num > verticals:
#                     if page == 'B%20-%20patron%20saint%20of%20mental%20illness%20-%20Google%20Search.html?q=patron+saint+of+mental+illness&btnK=Google+Search (CRC)':
#                         print(page)
#                         print(total_time)
                    total_time = obj.Sum
                    
#             print('\n')
            grouped_dict[page][info_type]['titles'] = title_list  
            grouped_dict[page][info_type]['description'] = descr_list
            grouped_dict[page][info_type]['verticals'] = vert_list
            grouped_dict[page][info_type]['total_time'] = total_time
    all_grouped_info.append(grouped_dict)
    print('Done')
with open('obj/'+ "rectangle_info" + '.pkl', 'wb+') as f:
    pickle.dump(rect_info, f, pickle.HIGHEST_PROTOCOL)

LOADING PREVIOUS ANSWERS 

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [150]:
# N = amount of participants that saw the rectangle
# Sum = total time spent on rectangle
# Mean = Sum / N

all_blended, all_non_blended = [], [] # participant -> page -> tf, ff -> category -> [obj]
for grouped_dict in all_grouped_info:
    blended, non_blended = {}, {}
    for page in grouped_dict.keys():
        page_name = page.split(' ')[0]
        if page_name == 'benefits':
            page_type = 'NB'
        elif page_name == 'black':
            page_type = 'B'
        elif page_name == 'facebook':
            page_type = 'B'
        elif page_name == 'feet':
            page_type = 'B'
        elif page_name == 'unicef':
            page_type = 'NB'
        elif page_name == 'why':
            page_type = 'NB'
        else:
            page_type = page.split('%')[0]

        if page_type == 'B':
            blended[page] = grouped_dict[page]
        if page_type == 'NB':
            non_blended[page] = grouped_dict[page]
    all_blended.append(blended)
    all_non_blended.append(non_blended)

In [151]:
def first_fix_per_category(list):
    ff_per_category = []
    for dict in list:
        info = {}
        for page in dict:
            for info_type in dict[page]:
                if info_type == 'Time to First Fixation':
                    for rectangle_type in dict[page][info_type]:
                        if rectangle_type != 'total_time':
                            if rectangle_type not in info:
                                info[rectangle_type] = []
                            lowest_mean = math.inf
                            for rectangle in dict[page][info_type][rectangle_type]:
                                if rectangle.Mean < lowest_mean and rectangle.Mean != 0:
                                    lowest_mean = rectangle.Mean
                            info[rectangle_type].append(lowest_mean)
                else:
                    continue
        ff_per_category.append(info)
    return ff_per_category

# participant -> lowest first fix per category per page
ff_blended = first_fix_per_category(all_blended)
ff_non_blended = first_fix_per_category(all_non_blended)

In [152]:
def get_ff_means(list):
    info = {}
    for dict in list:
        for category in dict:
            ff_list = dict[category]
            if category not in info:
                info[category] = []
            clean_ff_list = [x for x in ff_list if x!=math.inf]
            if clean_ff_list:
                mean = np.mean(np.array(clean_ff_list))
            else:
                mean = 0
            info[category].append(mean)
    return info

ff_blended_means = get_ff_means(ff_blended)
ff_non_blended_means = get_ff_means(ff_non_blended)

In [153]:
def first_fix_category_amount(list):
    count = [0, 0, 0]
    for dict in list:
        for i, key in enumerate(dict):
            if i == 0:
                arr = np.array(dict[key])
            else:
                cat = np.array(dict[key])
                arr = np.vstack((arr, cat))
        for i in range(len(arr[0,:])):
            if np.all(arr[:,i] == [math.inf, math.inf, math.inf]):
                continue
            result = np.where(arr[:, i] == np.amin(arr[:, i]))
            index = int(result[0][0])
            count[index] += 1
    return count

# titles, descriptions, verticals
ff_blended_amount = first_fix_category_amount(ff_blended)
ff_non_blended_amount = first_fix_category_amount(ff_non_blended)

In [154]:
def total_fix_per_link(list):
    info = {}
    for dict in list:
        for page in dict:
            for info_type in dict[page]:
                if info_type == 'Total Fixation Duration':
                    for category in dict[page][info_type]:
                        if category != 'total_time' and category != 'verticals':
                            if category not in info:
                                info[category] = {}
                            value_list = []
                            obj_list = []
                            for obj in dict[page][info_type][category]:
                                num = int(obj.get_Name().split(' ')[1])
                                value_list.append(num)
                                obj_list.append(obj)
                            
                            sorted_list = value_list.copy()
                            sorted_list.sort()
                            
                            for i, value in enumerate(sorted_list):
                                index = value_list.index(value)
                                obj = obj_list[index]
                                if i not in info[category]:
                                    info[category][i] = 0
                                info[category][i] += obj.Sum
                        else:
                            continue
                else:
                    continue
    return info

# participant -> total fixation time per category per index
tf_per_link_blended = total_fix_per_link(all_blended)
tf_per_link_non_blended = total_fix_per_link(all_non_blended)

In [65]:
def total_fix_per_category(list):
    tf_per_category = []
    for i, dict in enumerate(list):
        info = {}
        for page in dict:
            for info_type in dict[page]:
                if info_type == 'Total Fixation Duration':
                    for rectangle_type in dict[page][info_type]:
                        if rectangle_type not in info:
                            info[rectangle_type] = []
                        if rectangle_type != 'total_time':
                            total_sum = 0   
                            for rectangle in dict[page][info_type][rectangle_type]:
                                total_sum += rectangle.Sum
                            info[rectangle_type].append(total_sum)
                        else:
                            info[rectangle_type].append(dict[page][info_type][rectangle_type])
                else:
                    continue
        tf_per_category.append(info)
    return tf_per_category

# participant -> total fix per category per page
tf_blended = total_fix_per_category(all_blended)
tf_non_blended = total_fix_per_category(all_non_blended)

In [155]:
def get_tf_means(list):
    info = {}
    for dict in list:
        for category in dict:
            tf_list = dict[category]
            if category not in info:
                info[category] = []
            info[category].append(np.sum(tf_list)/6)
    return info

# mean per category per participant
tf_blended_means = get_tf_means(tf_blended)
tf_non_blended_means = get_tf_means(tf_non_blended)

In [73]:
def get_tf_percentage(list):
    info = {}
    for i, dict in enumerate(list):
        for category in dict:
            if category != 'total_time':
                tf_list = dict[category]
                if category not in info:
                    info[category] = []            
                cat_time = np.sum(tf_list)
                total_time = np.sum(dict['total_time'])
                percentage = cat_time/total_time
                info[category].append(percentage)
            else:
                continue
    return info

tf_blended_percentage = get_tf_percentage(tf_blended)
tf_non_blended_percentage = get_tf_percentage(tf_non_blended)
# tf_blended_percentage

for key in tf_blended_percentage:
    print(key)
    for item in tf_blended_percentage[key]:
        print(item)
    print('\n')

titles
0.21622653758759622
0.24217090385801407
0.09028347298983776
0.2051256506687255
0.13432360953687686
0.10261088899025947
0.23643434221632703
0.10361345640666696
0.12029206715894558
0.14713168418444986
0.19903882499055026
0.022687122268712227


description
0.053980256248687256
0.08531487699581991
0.15660545551791763
0.15758013499041118
0.19837340173795465
0.1831915635817211
0.22576269142650432
0.1916969246248742
0.26694443298543785
0.035017871677814345
0.2641251327465486
0.10878661087866108


verticals
0.4631761852933876
0.3871719317480984
0.5817079693349972
0.37682747627705404
0.41716876454095325
0.6115061582633429
0.2845321355359942
0.5435495865960891
0.33088568953426006
0.6587040379375023
0.3109599150422089
0.7814659848132651


